In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Generate our categorical variable list
df_cat = df.dtypes[df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
df[df_cat].nunique()


home_ownership          4
verification_status     3
issue_d                 3
loan_status             2
pymnt_plan              1
initial_list_status     2
next_pymnt_d            2
application_type        2
hardship_flag           1
debt_settlement_flag    1
dtype: int64

In [7]:
# Convert the string values into numerical ones using the get_dummies() method.
encoded_df = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "loan_status", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type", "hardship_flag", "debt_settlement_flag"])
encoded_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [8]:
df = df.merge(encoded_df,left_index=True, right_index=True)
df = df.drop(df_cat,1)
df.head()

,loan_amnt_x,int_rate_x,installment_x,annual_inc_x,dti_x,delinq_2yrs_x,inq_last_6mths_x,open_acc_x,pub_rec_x,revol_bal_x,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [10]:
# Create our target
y = df.loan_status_high_risk.values

# Create our features
X = df.drop(columns=["loan_status_high_risk", "loan_status_low_risk"]).values


In [11]:
X.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

In [15]:
# Check the balance of our target values
y = df["loan_status_high_risk"].value_counts()
y

0    68470
1      347
Name: loan_status_high_risk, dtype: int64

In [16]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Remove loan status target from features data
y = df.loan_status_high_risk.values
X = df.drop(columns=["loan_status_high_risk","loan_status_low_risk"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

rf_model = rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_test_scaled)

In [18]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)

In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,17104,14
Actual 1,55,32


In [20]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17118
           1       0.70      0.37      0.48        87

    accuracy                           1.00     17205
   macro avg       0.85      0.68      0.74     17205
weighted avg       1.00      1.00      1.00     17205



In [21]:
# List the features sorted in descending order by feature importance

# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances



array([6.30357947e-03, 5.91217323e-03, 9.61106179e-03, 6.73759748e-03,
       8.50347128e-03, 2.36648274e-03, 2.93612603e-03, 3.88032877e-03,
       1.32703222e-03, 6.53360848e-03, 5.66972442e-03, 1.09896705e-02,
       1.01511743e-02, 3.55852158e-02, 4.09849527e-02, 3.77266007e-02,
       2.49967466e-02, 6.00158326e-03, 0.00000000e+00, 0.00000000e+00,
       3.40569759e-02, 6.01058108e-04, 0.00000000e+00, 0.00000000e+00,
       3.48610129e-03, 6.62646994e-03, 2.54037212e-03, 2.90995347e-03,
       1.61554420e-03, 3.88261167e-03, 6.17119246e-03, 6.99919530e-03,
       5.87958093e-03, 2.15718266e-03, 3.36913281e-03, 7.56800547e-03,
       5.56664662e-03, 5.50401152e-03, 3.52106090e-03, 2.99494556e-03,
       3.85747998e-03, 3.72851390e-03, 7.12590529e-03, 6.85120733e-03,
       8.13182031e-03, 0.00000000e+00, 0.00000000e+00, 9.06182195e-03,
       8.16329434e-03, 5.99374515e-03, 4.14500252e-03, 3.28224021e-03,
       5.71969398e-03, 4.56352851e-03, 1.62757867e-03, 3.49503684e-03,
      

In [22]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

### Easy Ensemble AdaBoost Classifier

In [25]:
from imblearn.ensemble import EasyEnsembleClassifier

In [26]:
# Train the EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(random_state=42)

eec_model = eec_model.fit(X_train_scaled, y_train)

predictions = eec_model.predict(X_test_scaled)



In [27]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,15879,1239
Actual 1,9,78


In [29]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.93      0.96     17118
           1       0.06      0.90      0.11        87

    accuracy                           0.93     17205
   macro avg       0.53      0.91      0.54     17205
weighted avg       0.99      0.93      0.96     17205

